In [4]:
from pandas import isna

from eis1600.markdown.markdown_patterns import PARAGRAPH_UID_TAG_PATTERN
from eis1600.processing.preprocessing import get_yml_and_miu_df
from eis1600.corpus_analysis.text_methods import get_text_as_list_of_mius

In [13]:
infile = '../../OpenITI_EIS1600_Texts/data/0346Mascudi/0346Mascudi.TanbihWaIshraf/0346Mascudi.TanbihWaIshraf.Shamela0023718-ara1.EIS1600'

mius_list = get_text_as_list_of_mius(infile)

uid, miu_as_text, analyse_flag = mius_list[0]
yml_handler, df = get_yml_and_miu_df(miu_as_text)

In [14]:
paragraphs = []

tokens = []
curr_section_type = None
for section, token, tags in df.itertuples(index=False):
    if isna(section):
        if not isna(token):
            tokens.append(token)
    elif PARAGRAPH_UID_TAG_PATTERN.match(section):
        paragraphs.append((curr_section_type, ' '.join(tokens)))
        curr_section_type = PARAGRAPH_UID_TAG_PATTERN.match(section).group('cat')
        tokens = []

paragraphs.append((curr_section_type, ' '.join(tokens)))
paragraphs = paragraphs[1:]    # First element is (None, '') due to the MIU header

In [16]:
mergable_paragraphs = []
unsplitted = []
for cat, text in paragraphs:
    if cat == 'UNDEFINED':
        # Condition probably better to exclude only poetry but this way BOOKS, TEACHER, STUDENTS are kept (probably unneccessary)
        # Test for poetry
        mergable_paragraphs.append(text)
    else:
        unsplitted.append(('UNDEFINED', ' '.join(mergable_paragraphs)))
        mergable_paragraphs = []
        unsplitted.append((cat, text))
if mergable_paragraphs:
    unsplitted.append(('UNDEFINED', ' '.join(mergable_paragraphs)))
print(len(unsplitted))
print([cat for cat, text in unsplitted])

5
['UNDEFINED', 'POETRY', 'UNDEFINED', 'POETRY', 'UNDEFINED']
